# Header

In [ ]:
import numpy as np
from ipywidgets import widgets
from Functions.Properties_Functions import * 

# Equipment Functions

In [ ]:
def Cal_Compressor_Turbine(z,P1,T1,P2):
    # Heat capacity coeff. for mixture
    MW_ = np.sum(z*MW)
    HA_ = np.sum(z*MW/MW_*HA)
    HB_ = np.sum(z*MW/MW_*HB)
    HC_ = np.sum(z*MW/MW_*HC)
    HD_ = np.sum(z*MW/MW_*HD)


    Tr1 = T1/Tc
    alpha1 = (1 + k*(1 - np.sqrt(Tr1)))**2
    a_alpha1 = a*alpha1
    a_alphaij1 = np.sqrt(np.array([a_alpha1]).T.dot(np.array([a_alpha1])))*(1-binary_coeff)
    a_alphamix1 = np.sum(np.array([z]).T.dot(np.array([z]))*a_alphaij1)
    b1 = np.sum(z*b)

    A1 = a_alphamix1*P1/((R*T1)**2)
    B1 = b1*P1/(R*T1)

    Z1 = cardan(A1,B1)
    if np.size(Z1) > 2:
      Z1 = max(Z1)
    v1 = Z1 * R * T1 / P1


    T2 = T1
    Tr2 = T2/Tc

    alpha2 = (1 + k*(1 - np.sqrt(Tr2)))**2
    a_alpha2 = a*alpha2
    a_alphaij2 = np.sqrt(np.array([a_alpha2]).T.dot(np.array([a_alpha2])))*(1-binary_coeff)
    a_alphamix2 = np.sum(np.array([z]).T.dot(np.array([z]))*a_alphaij2)
    b2 = np.sum(z*b)

    A2 = a_alphamix2*P2/((R*T2)**2)
    B2 = b2*P2/(R*T2)

    Z2 = cardan(A2,B2)
    if np.size(Z2) > 2:
      Z2 = max(Z2)
    v2 = Z2 * R * T2 / P2

    ds = cp_Tdt(HA_,HB_,HC_,HD_,T1,T2,P1,P2) + sdep(P2,Z2,T2,z) - sdep(P1,Z1,T1,z)
    ds0 = ds
    T0 = T2

    T2 = T1 + 1.0
    Tr2 = T2/Tc

    alpha2 = (1 + k*(1 - np.sqrt(Tr2)))**2
    a_alpha2 = a*alpha2
    a_alphaij2 = np.sqrt(np.array([a_alpha2]).T.dot(np.array([a_alpha2])))*(1-binary_coeff)
    a_alphamix2 = np.sum(np.array([z]).T.dot(np.array([z]))*a_alphaij2)
    b2 = np.sum(z*b)

    A2 = a_alphamix2*P2/((R*T2)**2)
    B2 = b2*P2/(R*T2)

    Z2 = cardan(A2,B2)
    if np.size(Z2) > 2:
      Z2 = max(Z2)
    v2 = Z2 * R * T2 / P2

    ds = cp_Tdt(HA_,HB_,HC_,HD_,T1,T2,P1,P2) + sdep(P2,Z2,T2,z) - sdep(P1,Z1,T1,z)

    while abs(ds) > 1.0e-5:

      T00 = T2
      T2 = T2 - ds/((ds-ds0)/(T2-T0))
      ds0 = ds
      T0 = T00
      Tr2 = T2/Tc

      alpha2 = (1 + k*(1 - np.sqrt(Tr2)))**2
      a_alpha2 = a*alpha2
      a_alphaij2 = np.sqrt(np.array([a_alpha2]).T.dot(np.array([a_alpha2])))*(1-binary_coeff)
      a_alphamix2 = np.sum(np.array([z]).T.dot(np.array([z]))*a_alphaij2)
      b2 = np.sum(z*b)

      A2 = a_alphamix2*P2/((R*T2)**2)
      B2 = b2*P2/(R*T2)

      Z2 = cardan(A2,B2)
      if np.size(Z2) > 2:
        Z2 = max(Z2)
      v2 = Z2 * R * T2 / P2

      ds = cp_Tdt(HA_,HB_,HC_,HD_,T1,T2,P1,P2) + sdep(P2,Z2,T2,z) - sdep(P1,Z1,T1,z)
    print('T2 when 100% eff =',np.round(T2 - 273.15,4))
    dh = cpdt(HA_,HB_,HC_,HD_,T1,T2) + hdep(P2,v2,T2,z) - hdep(P1,v1,T1,z)
    print('Q =',np.round(dh,4),'kJ/kmol')
    print('Positive Q: Compressor, Negative Q: Turbine')
    
    return(np.round(T2 - 273.15,4),np.round(dh,4))

In [ ]:
result = Cal_Compressor_Turbine(np.array([0.0, 0.0, 0.0, 0.5, 0.5]),1.0e5, 100 + 273.15,10.0e5)
result[0],result[1]

T2 when 100% eff = 173.0905
Q = 7569.474 kJ/kmol
Positive Q: Compressor, Negative Q: Turbine


(173.0905, 7569.474)

# TEST 

In [ ]:
# N2, C1, C2, C3, C4
z = np.array([0.0, 0.0, 0.0, 0.5, 0.5])
P1 = 1.0e5
T1 = 100
P2 = 10.0e5
T2 , W = Cal_Compressor_Turbine(z,P1,T1+273.15,P2)
print('T2 =', T2, '℃' , "W=" , W)

T2 when 100% eff = 173.0905
Q = 7569.474 kJ/kmol
Positive Q: Compressor, Negative Q: Turbine
T2 = 173.0905 ℃ W= 7569.474


# Widget

In [ ]:
Label1 = widgets.Label("Composition")
wComp1 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='N2:',
    disabled=False
)
wComp2 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='C1:',
    disabled=False
)
wComp3 = widgets.BoundedFloatText(
    value=0.0,
    min=0,
    max=1.0,
    step=0.1,
    description='C2:',
    disabled=False
)
wComp4 = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='C3:',
    disabled=False
)
wComp5 = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='C4:',
    disabled=False
)
wCompSum = widgets.BoundedFloatText(
    value=wComp1.value+wComp2.value+wComp3.value+wComp4.value+wComp5.value,
    min=0,
    max=1.0,
    step=0.1,
    description='SUM:',
    disabled=False
)

Label2 = widgets.Label("Input")

wInputPressure = widgets.BoundedFloatText(
    value=1,
    min=0,
    max=300.0,
    step=0.1,
    description='P1 (bar) :',
    disabled=False
)
wInputTemperature = widgets.BoundedFloatText(
    value=100,
    min=-500,
    max=500.0,
    step=0.1,
    description='T1 (C) :',
    disabled=False
)

wOutputPressure = widgets.BoundedFloatText(
    value=10,
    min=0,
    max=300.0,
    step=0.1,
    description='P2 (bar) :',
    disabled=False
)

Label3 = widgets.Label("< ### Result ### >")

wOutputTemperature = widgets.BoundedFloatText(
    value=0,
    min=-500,
    max=500.0,
    step=0.1,
    description='T2 (C) :',
    disabled=False
)

wOutputEnergy = widgets.BoundedFloatText(
    value=0,
    min=-1000000,
    max=1000000,
    step=0.1,
    description='W(Q) (KJ/Kmol) :',
    disabled=False
)


btn = widgets.Button(description = "Calculate")
lbl3 = widgets.Label()

def Calculate_Compressor_Turbine(a):
    z = np.array([wComp1.value,wComp2.value,wComp3.value,wComp4.value,wComp5.value])
    P1 = wInputPressure.value * 1e5
    T1 = wInputTemperature.value + 273.15
    P2 = wOutputPressure.value * 1e5
    tTemp,tEnergy = Cal_Compressor_Turbine(z,P1,T1,P2)
    wOutputTemperature.value = tTemp
    wOutputEnergy.value = tEnergy


# Compressor / Turbine ( Vapor )

In [ ]:
display(widgets.Label("### Compressor / Turbine (Vapor) Calculation ###"))
display(Label1,wComp1,wComp2,wComp3,wComp4,wComp5)
display(Label2,wInputPressure,wInputTemperature,wOutputPressure)
display(btn)
display(Label3,wOutputTemperature,wOutputEnergy)
btn.on_click(Calculate_Compressor_Turbine)

Label(value='### Compressor / Turbine (Vapor) Calculation ###')

Label(value='Composition')

BoundedFloatText(value=0.0, description='N2:', max=1.0, step=0.1)

BoundedFloatText(value=0.0, description='C1:', max=1.0, step=0.1)

BoundedFloatText(value=0.0, description='C2:', max=1.0, step=0.1)

...

BoundedFloatText(value=10.0, description='P2 (bar) :', max=300.0, step=0.1)

Button(description='Calculate', style=ButtonStyle())

Label(value='< ### Result ### >')

BoundedFloatText(value=0.0, description='T2 (C) :', max=500.0, min=-500.0, step=0.1)

BoundedFloatText(value=0.0, description='W(Q) (KJ/Kmol) :', max=1000000.0, min=-1000000.0, step=0.1)